In [ ]:
# Isolate the loop process using numba


import numpy as np
import scipy.stats as sts
from numba import jit

@jit(nopython=True)
def simulate_health_index(eps_mat: 'f8[:, :]', S: 'int64', T: 'int64', rho: 'float64', mu: 'float64', z_0: 'float64') -> 'f8[:, :]':
    z_mat = np.zeros((T, S), dtype=np.float64)
    for s_ind in range(S):
        z_tm1 = z_0
        for t_ind in range(T):
            e_t = eps_mat[t_ind, s_ind]
            z_t = rho * z_tm1 + (1 - rho) * mu + e_t
            z_mat[t_ind, s_ind] = z_t
            z_tm1 = z_t
    return z_mat

In [ ]:
# Run and evaluate code execution time


import time

# Original code execution time
start_time_original = time.time()

# Set model parameters
rho = 0.5
mu = 3.0
sigma = 1.0
z_0 = mu

# Set simulation parameters, draw all idiosyncratic random shocks,
# and create empty containers
S = 1000 # Set the number of lives to simulate
T = int(4160) # Set the number of periods for each simulatio
np.random.seed(25)
eps_mat = sts.norm.rvs(loc=0, scale=sigma, size=(T, S))
z_mat = np.zeros((T, S))
    
for s_ind in range(S):
    z_tm1 = z_0
    for t_ind in range(T):
        e_t = eps_mat[t_ind, s_ind]
        z_t = rho * z_tm1 + (1 - rho) * mu + e_t
        z_mat[t_ind, s_ind] = z_t
        z_tm1 = z_t

end_time_original = time.time()
original_execution_time = end_time_original - start_time_original


# Numba-accelerated execution time
start_time_numba = time.time()

# Set model parameters
rho = 0.5
mu = 3.0
sigma = 1.0
z_0 = mu

# Set simulation parameters, draw all idiosyncratic random shocks, and create empty containers
S = 1000  # Set the number of lives to simulate
T = 4160  # Set the number of periods for each simulation
np.random.seed(25)
eps_mat = sts.norm.rvs(loc=0, scale=sigma, size=(T, S))

z_mat = simulate_health_index(eps_mat, S, T, rho, mu, z_0)
end_time_numba = time.time()
numba_execution_time = end_time_numba - start_time_numba


print(f"Original Execution Time: {original_execution_time} seconds")
print(f"Numba-accelerated Execution Time: {numba_execution_time} seconds")

# Calculate and print the speedup
speedup = original_execution_time / numba_execution_time
print(f"Speedup: {speedup}x")